# The notebook is to demonstrate Indicating problem

Generate $|z'\rangle$ from quantum part

2-regular graph with 4 nodes(a ring with $Z_4=Z_0$)
    \begin{eqnarray}
        H_c=-\sum_{i=0}^3\frac{Z_iZ_{i+1}}{2},\quad  H_b =\sum_{i=0}^3 X_i .
    \end{eqnarray}

In [1]:
import numpy as np
from scipy.optimize import minimize
from scipy.linalg import expm


# Tensor product function
def tensor_product(*args):
    result = np.array([1])
    for arg in args:
        result = np.kron(result, arg)
    return result

# Unitary evolution operators
def unitary(H, theta):
    return expm(-1j*H*theta)


# Full circuit
def apply_circuit(gamma, beta, p, initial_state, H_c,H_b):
    state = initial_state
    # Define the Hamiltonians H_c and H_b
    for _ in range(p):
        state = unitary(H_c, gamma) @ state
        state = unitary(H_b, beta) @ state
    return state


# Cost function
def cost_function(params, p, initial_state, H_c, H_b):
    gamma, beta = params
  
    final_state = apply_circuit(gamma, beta, p, initial_state, H_c, H_b)
    expectation_value = np.real(np.vdot(final_state, H_c @ final_state))
    
    # Increment the counter and print every 10 iterations
    global iteration_counter
    iteration_counter += 1
    if iteration_counter % 10 == 0:
        print(f'Iteration: {iteration_counter}, gamma: {gamma:.4f}, beta: {beta:.4f}, cost: {expectation_value:.4f}')

    
    
    return expectation_value


def find_ground_energy(hamiltonian):
    """
    Find the ground energy of a Hamiltonian.
    
    Parameters:
    hamiltonian (np.ndarray): The Hamiltonian matrix.
    
    Returns:
    float: The ground energy (lowest eigenvalue) of the Hamiltonian.
    """
    # Calculate the eigenvalues of the Hamiltonian
    eigenvalues = np.linalg.eigvalsh(hamiltonian)
    
    # Return the smallest eigenvalue
    return np.min(eigenvalues)

Pre-train with an initial PQC, circuit formulates like this: $[e ^{-i H_b \beta}e^{-iH_c \gamma}]$

In [2]:
# Define Pauli matrices
I = np.eye(2)
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])

# Initial state |+>^4 = (|0> + |1>)/sqrt(2) ⊗ 4
initial_state = tensor_product(np.array([1, 1]) / np.sqrt(2), 
                               np.array([1, 1]) / np.sqrt(2),
                               np.array([1, 1]) / np.sqrt(2),
                               np.array([1, 1]) / np.sqrt(2))
# Define the Hamiltonians H_c and H_b
H_c = -(tensor_product(Z, Z, I, I)+tensor_product(I, Z, Z, I)+tensor_product(I, I, Z, Z)+tensor_product(Z, I, I, Z))
H_b = tensor_product(X, I, I, I)+tensor_product(I, X, I, I)+tensor_product(I, I, X, I)+tensor_product(I, I, I, X)  

# Global iteration counter
iteration_counter = 0

# Optimize using scipy
p = 1  # Number of layers
initial_params = np.random.rand(2)  # Initial guess for gamma and beta

result = minimize(cost_function, initial_params, args=(p,initial_state, H_c, H_b,), method='COBYLA')
optimal_gamma, optimal_beta = result.x

print(f'Optimal gamma: {optimal_gamma}, Optimal beta: {optimal_beta}')


temp_eig3 = find_ground_energy(H_c)
print(temp_eig3)

Iteration: 10, gamma: 1.1958, beta: 2.0984, cost: -1.7113
Iteration: 20, gamma: 1.1754, beta: 1.9560, cost: -1.9990
Iteration: 30, gamma: 1.1777, beta: 1.9632, cost: -2.0000
Optimal gamma: 1.1782219840879053, Optimal beta: 1.9634838418255447
-4.0


Then, using quantum gradient algorithm, we can obtain an updated $|z'\rangle$

In [3]:
def fidelity(state1, state2):

    state1 = np.asarray(state1)
    state2 = np.asarray(state2)
    
#     print(np.vdot(state1, state1))
    # Normalize the states
    state1 = state1 / np.linalg.norm(state1)
    state2 = state2 / np.linalg.norm(state2)
    
    # Calculate the fidelity
    fidelity_value = np.abs(np.vdot(state1, state2))**2
    
    return fidelity_value


def gradient(hamiltonian, xi):
    """
    Compute the gradient of the Hamiltonian with respect to a parameter.

    Parameters:
    hamiltonian (numpy.ndarray): The Hamiltonian matrix.
    xi (float): The scalar value representing the parameter with respect to which 
                the gradient is being computed.

    Returns:
    numpy.ndarray: The gradient matrix, which is the Hamiltonian scaled by the parameter xi.
    """
    
    # Compute the gradient by scaling the Hamiltonian with the parameter xi
    gradient = xi * hamiltonian
    
    # Return the resulting gradient matrix
    return gradient

def PQC_RL(state_in, state_out):
    """
    Compute parameterized quantum circuit that tranform state_in to state_out.

    Parameters:
    state_in (numpy.ndarray): the initial state.
    state_out (numpy.ndarray): the end state.
    max_depth (int): maximum depth of the circuit.

    Returns:
    ...: the learned policy.
    """
    # Parameters
    gamma = 0.99
    n_epochs = 4
    clip_range = 0.2
    learning_rate = 0.0001
    policy_kwargs = dict(optimizer_class=optim.Adam)

    # Agent
    ppo_model = PPO("MlpPolicy",
                    env,
                    gamma=gamma,
                    n_epochs=n_epochs,
                    clip_range=clip_range,
                    learning_rate=learning_rate,
                    policy_kwargs=policy_kwargs,
                    tensorboard_log='logs/')
    ppo_model.learn(total_timesteps=20000)

    return ppo_model



In [10]:
import gym
import numpy as np
import torch.optim as optim
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
import qas_gym

xi=0.2

state_in=apply_circuit(optimal_gamma, optimal_beta, p, initial_state, H_c,H_b)

state_out=state_in-gradient(H_c, xi)@state_in

state_out = state_out/np.linalg.norm(state_out)

# # print(state_out)

print(fidelity(state_out, state_in))


# fidelity(state_out, state_out_pqc)

# expectation_value = np.real(np.vdot(state_out, H_c @ state_out))
# print(expectation_value)



env_name = 'BasicFourQubit-v0' # 四比特环境-无噪声
fidelity_threshold = 0.92
reward_penalty = 0.05
# max_timesteps = 5
max_depth = 20
env = gym.make(env_name, target = state_out,
        fidelity_threshold=fidelity_threshold,
        reward_penalty=reward_penalty,
        max_timesteps=max_depth,
        initial = state_in)
ppo_model = PQC_RL(state_in, state_out)


# simulate to get fidelity
state = env.reset()
print(state)
done = False
while not done:
    action = ppo_model.predict(state)
    state, reward, done, info = env.step(action[0])
    #展示当前的线路 和 state
    env.render()
    # print(state)
    print(info['fidelity'])


0.9074257211990331
number of gates:1,  number of param gates:0
Parameters:  None
number of gates:2,  number of param gates:1
Parameters:  [0.10015841]
number of gates:3,  number of param gates:1
Parameters:  [0.4158881]
number of gates:4,  number of param gates:2
Parameters:  [ 0.42968008 -0.23947925]
number of gates:5,  number of param gates:2
Parameters:  [ 1.02243509 -0.08744872]
number of gates:6,  number of param gates:2
Parameters:  [ 0.12496112 -0.29452601]
number of gates:7,  number of param gates:2
Parameters:  [ 1.02241828 -0.08752505]
number of gates:8,  number of param gates:3
Parameters:  [ 1.00986836 -0.07604251 -0.1776385 ]
number of gates:9,  number of param gates:4
Parameters:  [ 1.00982291 -0.07620387 -0.38487469  0.20734087]
number of gates:10,  number of param gates:5
Parameters:  [ 1.01561388 -0.04223481 -0.47744003  0.30217603 -0.04554353]
number of gates:11,  number of param gates:5
Parameters:  [ 0.73027834  0.05529885 -0.63804929  0.13373215 -0.48643071]
number

C:\Users\Mac\AppData\Roaming\Python\Python311\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
C:\Users\Mac\AppData\Roaming\Python\Python311\site-packages\gym\utils\passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
C:\Users\Mac\AppData\Roaming\Python\Python311\site-packages\gym\utils\passive_env_checker.py:187: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.warn(
C:\Users\Mac\AppData\Roaming\Python\Python311\site-packages\

Parameters:  [ 0.33055813 -0.03220472 -0.62123977 -0.31665139  0.06846946 -0.23596952
  0.0578921  -0.16175138]
number of gates:17,  number of param gates:9
Parameters:  [ 0.32675729 -0.12993688 -0.69836827 -0.25891007  0.14586449 -0.22727678
  0.08235563 -0.09369309 -0.12644209]
number of gates:18,  number of param gates:9
Parameters:  [ 1.06575407 -0.55338506 -0.10688299 -0.25233014 -0.04767846  0.53997253
  0.23791037  0.2397092  -0.61963163]
number of gates:19,  number of param gates:9
Parameters:  [ 0.19322997 -0.22256693  1.4781824  -0.94798011 -0.313089    0.85525405
  1.36862601 -1.04045662  0.41121049]
number of gates:20,  number of param gates:10
Parameters:  [-1.57032072e+00 -2.04286625e-01  1.26325167e+00 -1.41710239e+00
 -3.16816185e-04  7.66014606e-01  1.65468142e+00 -1.45044713e+00
  5.23985615e-01  1.57043865e+00]
number of gates:1,  number of param gates:1
Parameters:  [6.51893738e-05]
number of gates:2,  number of param gates:1
Parameters:  [-0.06892661]
number of gat

C:\Users\Mac\AppData\Roaming\Python\Python311\site-packages\gym\utils\passive_env_checker.py:272: UserWarning: WARN: No render modes was declared in the environment (env.metadata['render_modes'] is None or not defined), you may have trouble when calling `.render()`.
  logger.warn(


Parameters:  [-0.15374251 -0.74340699  2.07985645  0.22973726 -1.72290342 -0.53732321
  0.24078846 -0.53424165 -1.41728827 -1.02742224]

                                              ┌───────────┐
0: ───I─────────────────@─────────────────@──────────────X────Rz(0.077π)─────────────────────────────────────────────
                        │                 │              │
1: ───I───Ry(-0.049π)───X───Ry(-0.237π)───X────Ry(0.073π)┼────@────────────Ry(-0.548π)───Rz(-0.451π)─────────────────
                                                         │    │
2: ───I───Rz(0.662π)─────────────────────────────────────┼────X────────────Ry(-0.171π)───Rz(-0.17π)────Ry(-0.327π)───
                                                         │
3: ───I──────────────────────────────────────────────────@───────────────────────────────────────────────────────────
                                              └───────────┘
0.25808321183450683
number of gates:15,  number of param gates:11
Parameters:  [ 0.570120

In [11]:
print(info['fidelity'])

0.20444842658732248


In [6]:
state_in

array([-1.24931844e-01-4.99999969e-01j,  1.15666681e-05-1.25062369e-01j,
        1.15666681e-05-1.25062369e-01j,  1.24943417e-01-2.50000000e-01j,
        1.15666681e-05-1.25062369e-01j, -1.24931844e-01-3.08520560e-08j,
        1.24943417e-01-2.50000000e-01j,  1.15666681e-05-1.25062369e-01j,
        1.15666681e-05-1.25062369e-01j,  1.24943417e-01-2.50000000e-01j,
       -1.24931844e-01-3.08520559e-08j,  1.15666681e-05-1.25062369e-01j,
        1.24943417e-01-2.50000000e-01j,  1.15666681e-05-1.25062369e-01j,
        1.15666681e-05-1.25062369e-01j, -1.24931844e-01-4.99999969e-01j])

In [8]:
fidelity(state_out, state_in)

0.9074257211990331